Análisis completo de detección de fraude en tarjetas de crédito
Incluye EDA completo, preprocesamiento, modelado con XGBoost y métricas detalladas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import (classification_report, confusion_matrix, 
                            roc_auc_score, roc_curve, precision_recall_curve, 
                            average_precision_score, f1_score, precision_score, recall_score)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configuración de estilo para gráficos
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

# Cargar datos
print("="*80)
print("ANÁLISIS DE DETECCIÓN DE FRAUDE - DATASET CREDIT CARD 2023")
print("="*80)
print("\n1. CARGA DE DATOS")
print("-"*80)

df = pd.read_csv('Archivos_CSV/creditcard_2023.csv')
print(f"Shape del dataset: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")

ANÁLISIS DE DETECCIÓN DE FRAUDE - DATASET CREDIT CARD 2023

1. CARGA DE DATOS
--------------------------------------------------------------------------------
Shape del dataset: (568630, 31)
Columnas: ['id', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']


In [3]:
# ============================================================================
# 2. ANÁLISIS EXPLORATORIO DE DATOS (EDA)
# ============================================================================
print("\n" + "="*80)
print("2. ANÁLISIS EXPLORATORIO DE DATOS (EDA)")
print("="*80)

# 2.1 Información básica
print("\n2.1. Información básica del dataset")
print("-"*80)
print(df.info())
print(f"\nValores faltantes por columna:\n{df.isnull().sum()}")
print(f"\nTotal de valores faltantes: {df.isnull().sum().sum()}")

# 2.2 Estadísticas descriptivas
print("\n2.2. Estadísticas descriptivas")
print("-"*80)
print(df.describe())

# 2.3 Análisis de la variable objetivo
print("\n2.3. Análisis de la variable objetivo (Class)")
print("-"*80)
class_distribution = df['Class'].value_counts()
class_percentage = df['Class'].value_counts(normalize=True) * 100
print(f"Distribución de clases:\n{class_distribution}")
print(f"\nPorcentajes:\n{class_percentage}")

# Verificar si hay duplicados
print(f"\nDuplicados en el dataset: {df.duplicated().sum()}")

# 2.4 Análisis de la variable Amount
print("\n2.4. Análisis de la variable Amount")
print("-"*80)
print(f"Estadísticas de Amount:\n{df['Amount'].describe()}")
print(f"\nSkewness de Amount: {df['Amount'].skew():.4f}")
print(f"Kurtosis de Amount: {df['Amount'].kurtosis():.4f}")


2. ANÁLISIS EXPLORATORIO DE DATOS (EDA)

2.1. Información básica del dataset
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568630 entries, 0 to 568629
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      568630 non-null  int64  
 1   V1      568630 non-null  float64
 2   V2      568630 non-null  float64
 3   V3      568630 non-null  float64
 4   V4      568630 non-null  float64
 5   V5      568630 non-null  float64
 6   V6      568630 non-null  float64
 7   V7      568630 non-null  float64
 8   V8      568630 non-null  float64
 9   V9      568630 non-null  float64
 10  V10     568630 non-null  float64
 11  V11     568630 non-null  float64
 12  V12     568630 non-null  float64
 13  V13     568630 non-null  float64
 14  V14     568630 non-null  float64
 15  V15     568630 non-null  float64
 16  V16     568630 non-null  float64
 17  V17

In [4]:
# ============================================================================
# 3. VISUALIZACIONES EDA
# ============================================================================
print("\n" + "="*80)
print("3. GENERANDO VISUALIZACIONES EDA")
print("="*80)

# Crear directorio para guardar gráficos
import os
os.makedirs('graficos_eda', exist_ok=True)

# 3.1 Distribución de clases
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
class_distribution.plot(kind='bar', ax=axes[0], color=['skyblue', 'coral'])
axes[0].set_title('Distribución de Clases (Conteo)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Clase (0=Normal, 1=Fraude)', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].tick_params(axis='x', rotation=0)

class_percentage.plot(kind='bar', ax=axes[1], color=['skyblue', 'coral'])
axes[1].set_title('Distribución de Clases (Porcentaje)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Clase (0=Normal, 1=Fraude)', fontsize=12)
axes[1].set_ylabel('Porcentaje (%)', fontsize=12)
axes[1].tick_params(axis='x', rotation=0)
for i, v in enumerate(class_percentage):
    axes[1].text(i, v + 0.5, f'{v:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('graficos_eda/01_distribucion_clases.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 1 guardado: 01_distribucion_clases.png")

# 3.2 Distribución de Amount por clase
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Boxplot
df.boxplot(column='Amount', by='Class', ax=axes[0, 0])
axes[0, 0].set_title('Distribución de Amount por Clase (Boxplot)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Clase', fontsize=12)
axes[0, 0].set_ylabel('Amount', fontsize=12)

# Histograma
df[df['Class']==0]['Amount'].hist(bins=50, ax=axes[0, 1], alpha=0.7, label='Normal', color='skyblue')
df[df['Class']==1]['Amount'].hist(bins=50, ax=axes[0, 1], alpha=0.7, label='Fraude', color='coral')
axes[0, 1].set_title('Distribución de Amount por Clase (Histograma)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Amount', fontsize=12)
axes[0, 1].set_ylabel('Frecuencia', fontsize=12)
axes[0, 1].legend()
axes[0, 1].set_yscale('log')

# Distribución logarítmica
df_normal = df[df['Class']==0]['Amount']
df_fraud = df[df['Class']==1]['Amount']
axes[1, 0].hist(np.log1p(df_normal), bins=50, alpha=0.7, label='Normal', color='skyblue')
axes[1, 0].hist(np.log1p(df_fraud), bins=50, alpha=0.7, label='Fraude', color='coral')
axes[1, 0].set_title('Distribución de Amount (Log Transform)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('log(Amount + 1)', fontsize=12)
axes[1, 0].set_ylabel('Frecuencia', fontsize=12)
axes[1, 0].legend()

# Estadísticas por clase
stats_by_class = df.groupby('Class')['Amount'].agg(['mean', 'median', 'std', 'min', 'max'])
stats_by_class.plot(kind='bar', ax=axes[1, 1], width=0.8)
axes[1, 1].set_title('Estadísticas de Amount por Clase', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Clase', fontsize=12)
axes[1, 1].set_ylabel('Valor', fontsize=12)
axes[1, 1].legend(['Media', 'Mediana', 'Desv. Estándar', 'Mínimo', 'Máximo'])
axes[1, 1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig('graficos_eda/02_distribucion_amount.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 2 guardado: 02_distribucion_amount.png")

# 3.3 Análisis de características V1-V28
print("\nGenerando análisis de características V1-V28...")
v_columns = [col for col in df.columns if col.startswith('V')]

# Correlación con la variable objetivo
correlations = df[v_columns + ['Amount']].corrwith(df['Class']).sort_values(ascending=False)
top_correlations = correlations.head(10)

fig, ax = plt.subplots(figsize=(12, 8))
top_correlations.plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Top 10 Características más Correlacionadas con Fraude', fontsize=14, fontweight='bold')
ax.set_xlabel('Correlación con Class', fontsize=12)
plt.tight_layout()
plt.savefig('graficos_eda/03_correlacion_top_features.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 3 guardado: 03_correlacion_top_features.png")

# 3.4 Distribución de características más importantes
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
top_features = correlations.abs().head(6).index.tolist()

for idx, feature in enumerate(top_features):
    row = idx // 3
    col = idx % 3
    df[df['Class']==0][feature].hist(bins=50, ax=axes[row, col], alpha=0.6, label='Normal', color='skyblue', density=True)
    df[df['Class']==1][feature].hist(bins=50, ax=axes[row, col], alpha=0.6, label='Fraude', color='coral', density=True)
    axes[row, col].set_title(f'Distribución de {feature}', fontsize=12, fontweight='bold')
    axes[row, col].set_xlabel(feature, fontsize=10)
    axes[row, col].set_ylabel('Densidad', fontsize=10)
    axes[row, col].legend()

plt.tight_layout()
plt.savefig('graficos_eda/04_distribucion_top_features.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 4 guardado: 04_distribucion_top_features.png")

# 3.5 Matriz de correlación de características más importantes
fig, ax = plt.subplots(figsize=(12, 10))
top_15_features = correlations.abs().head(15).index.tolist()
correlation_matrix = df[top_15_features + ['Class']].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlación - Top 15 Características', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('graficos_eda/05_matriz_correlacion.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 5 guardado: 05_matriz_correlacion.png")

# 3.6 Análisis de outliers usando IQR
print("\n3.6. Análisis de outliers")
print("-"*80)
Q1 = df[v_columns].quantile(0.25)
Q3 = df[v_columns].quantile(0.75)
IQR = Q3 - Q1
outliers = ((df[v_columns] < (Q1 - 1.5 * IQR)) | (df[v_columns] > (Q3 + 1.5 * IQR))).sum(axis=1)
df['outlier_count'] = outliers
print(f"Transacciones con outliers: {(df['outlier_count'] > 0).sum()}")
print(f"Fraudes con outliers: {df[df['Class']==1]['outlier_count'].gt(0).sum()}")


3. GENERANDO VISUALIZACIONES EDA
[OK] Grafico 1 guardado: 01_distribucion_clases.png
[OK] Grafico 2 guardado: 02_distribucion_amount.png

Generando análisis de características V1-V28...
[OK] Grafico 3 guardado: 03_correlacion_top_features.png
[OK] Grafico 4 guardado: 04_distribucion_top_features.png
[OK] Grafico 5 guardado: 05_matriz_correlacion.png

3.6. Análisis de outliers
--------------------------------------------------------------------------------
Transacciones con outliers: 241919
Fraudes con outliers: 168784


In [5]:
# ============================================================================
# 4. PREPROCESAMIENTO DE DATOS
# ============================================================================
print("\n" + "="*80)
print("4. PREPROCESAMIENTO DE DATOS")
print("="*80)

# Separar características y variable objetivo
X = df.drop(['Class', 'id', 'outlier_count'], axis=1, errors='ignore')
y = df['Class']

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"Distribución de clases en y: {y.value_counts().to_dict()}")

# División train-test estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDivisión train-test:")
print(f"Train: {X_train.shape[0]} muestras ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Test: {X_test.shape[0]} muestras ({X_test.shape[0]/len(df)*100:.1f}%)")
print(f"\nDistribución en train: {y_train.value_counts().to_dict()}")
print(f"Distribución en test: {y_test.value_counts().to_dict()}")

# Escalado robusto (mejor para datos con outliers)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\n[OK] Escalado completado (RobustScaler)")


4. PREPROCESAMIENTO DE DATOS
Shape de X: (568630, 29)
Shape de y: (568630,)
Distribución de clases en y: {0: 284315, 1: 284315}

División train-test:
Train: 454904 muestras (80.0%)
Test: 113726 muestras (20.0%)

Distribución en train: {0: 227452, 1: 227452}
Distribución en test: {1: 56863, 0: 56863}

[OK] Escalado completado (RobustScaler)


In [6]:
# ============================================================================
# 5. MANEJO DE CLASES DESBALANCEADAS
# ============================================================================
print("\n" + "="*80)
print("5. MANEJO DE CLASES DESBALANCEADAS")
print("="*80)

# Verificar desbalance
class_counts = y_train.value_counts()
if len(class_counts) > 1:
    imbalance_ratio = class_counts.iloc[0] / class_counts.iloc[1]
    print(f"Ratio de desbalance (Normal:Fraude): {imbalance_ratio:.1f}:1")
    
    # Aplicar SMOTE solo si hay desbalance significativo (> 5%)
    if imbalance_ratio > 1.05 or imbalance_ratio < 0.95:
        print("\nAplicando SMOTE para balancear...")
        smote = SMOTE(random_state=42, sampling_strategy=0.5)  # 50% de fraude
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
        print(f"Despues de SMOTE:")
        print(f"Train balanced: {X_train_balanced.shape[0]} muestras")
        print(f"Distribucion: {pd.Series(y_train_balanced).value_counts().to_dict()}")
    else:
        print("\nDataset balanceado - no se aplica SMOTE")
        X_train_balanced = X_train_scaled
        y_train_balanced = y_train
        imbalance_ratio = 1.0
else:
    print("\nSolo una clase presente")
    X_train_balanced = X_train_scaled
    y_train_balanced = y_train
    imbalance_ratio = 1.0


5. MANEJO DE CLASES DESBALANCEADAS
Ratio de desbalance (Normal:Fraude): 1.0:1

Dataset balanceado - no se aplica SMOTE


In [7]:
# ============================================================================
# 6. ENTRENAMIENTO DEL MODELO XGBOOST
# ============================================================================
print("\n" + "="*80)
print("6. ENTRENAMIENTO DEL MODELO XGBOOST")
print("="*80)

# Configuración de XGBoost optimizado para detección de fraude
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',  # AUC-PR es mejor para clases desbalanceadas
    max_depth=6,
    learning_rate=0.1,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=1,
    gamma=0.1,
    scale_pos_weight=1.0 if imbalance_ratio == 1.0 else imbalance_ratio,  # Maneja desbalance
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)

print("Entrenando modelo XGBoost...")
xgb_model.fit(
    X_train_balanced, y_train_balanced,
    eval_set=[(X_train_scaled, y_train), (X_test_scaled, y_test)],
    verbose=False
)

print("[OK] Modelo entrenado")

# Importancia de características
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 15 características más importantes:")
print(feature_importance.head(15))

# Visualización de importancia
fig, ax = plt.subplots(figsize=(12, 10))
top_20_features = feature_importance.head(20)
sns.barplot(data=top_20_features, x='importance', y='feature', ax=ax, palette='viridis')
ax.set_title('Top 20 Características más Importantes (XGBoost)', fontsize=14, fontweight='bold')
ax.set_xlabel('Importancia', fontsize=12)
ax.set_ylabel('Característica', fontsize=12)
plt.tight_layout()
plt.savefig('graficos_eda/06_importancia_features.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 6 guardado: 06_importancia_features.png")


6. ENTRENAMIENTO DEL MODELO XGBOOST
Entrenando modelo XGBoost...
[OK] Modelo entrenado

Top 15 características más importantes:
   feature  importance
13     V14    0.388152
9      V10    0.250411
3       V4    0.081897
16     V17    0.032233
11     V12    0.022614
2       V3    0.021544
7       V8    0.018789
0       V1    0.015276
1       V2    0.012711
10     V11    0.012435
17     V18    0.010548
12     V13    0.009781
8       V9    0.009294
20     V21    0.009172
19     V20    0.008808
[OK] Grafico 6 guardado: 06_importancia_features.png


In [8]:
# ============================================================================
# 7. PREDICCIONES Y MÉTRICAS
# ============================================================================
print("\n" + "="*80)
print("7. PREDICCIONES Y MÉTRICAS")
print("="*80)

# Predicciones
y_pred = xgb_model.predict(X_test_scaled)
y_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

# 7.1 Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("\n7.1. Matriz de Confusión:")
print(cm)

# Visualización de matriz de confusión
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
            xticklabels=['Normal', 'Fraude'], yticklabels=['Normal', 'Fraude'])
ax.set_title('Matriz de Confusión', fontsize=14, fontweight='bold')
ax.set_ylabel('Clase Real', fontsize=12)
ax.set_xlabel('Clase Predicha', fontsize=12)
plt.tight_layout()
plt.savefig('graficos_eda/07_matriz_confusion.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 7 guardado: 07_matriz_confusion.png")

# 7.2 Reporte de clasificación
print("\n7.2. Reporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Fraude']))

# 7.3 Métricas específicas para fraude
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)
auc_pr = average_precision_score(y_test, y_pred_proba)

print("\n7.3. Métricas Detalladas:")
print("-"*80)
print(f"Precision (Fraude): {precision:.4f}")
print(f"Recall (Sensibilidad): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"AUC-PR (Precision-Recall): {auc_pr:.4f}")

# Cálculo de métricas adicionales
TN, FP, FN, TP = cm.ravel()
specificity = TN / (TN + FP)
accuracy = (TP + TN) / (TP + TN + FP + FN)

print(f"\nAccuracy: {accuracy:.4f}")
print(f"Specificity (Especificidad): {specificity:.4f}")
print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")


7. PREDICCIONES Y MÉTRICAS

7.1. Matriz de Confusión:
[[56800    63]
 [    0 56863]]
[OK] Grafico 7 guardado: 07_matriz_confusion.png

7.2. Reporte de Clasificación:
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     56863
      Fraude       1.00      1.00      1.00     56863

    accuracy                           1.00    113726
   macro avg       1.00      1.00      1.00    113726
weighted avg       1.00      1.00      1.00    113726


7.3. Métricas Detalladas:
--------------------------------------------------------------------------------
Precision (Fraude): 0.9989
Recall (Sensibilidad): 1.0000
F1-Score: 0.9994
AUC-ROC: 1.0000
AUC-PR (Precision-Recall): 1.0000

Accuracy: 0.9994
Specificity (Especificidad): 0.9989
True Positives (TP): 56863
True Negatives (TN): 56800
False Positives (FP): 63
False Negatives (FN): 0


In [10]:
# ============================================================================
# 8. GUARDAR MODELO Y SCALER
# ============================================================================
print("\n" + "="*80)
print("8. GUARDANDO MODELO Y SCALER")
print("="*80)

import joblib
import os

# Crear carpeta si no existe
os.makedirs('Modelo_Entrenado', exist_ok=True)

# Guardar el modelo XGBoost
modelo_path = 'Modelo_Entrenado/modelo_xgboost_fraude.pkl'
joblib.dump(xgb_model, modelo_path)
print(f"[OK] Modelo guardado en: {modelo_path}")

# Guardar el scaler
scaler_path = 'Modelo_Entrenado/scaler_robust.pkl'
joblib.dump(scaler, scaler_path)
print(f"[OK] Scaler guardado en: {scaler_path}")

# Guardar lista de columnas (features) para validar inputs
features_path = 'Modelo_Entrenado/features_names.pkl'
feature_names = X_train.columns.tolist()
joblib.dump(feature_names, features_path)
print(f"[OK] Nombres de features guardados en: {features_path}")

# Guardar metadatos del modelo en un archivo de texto
metadata_path = 'Modelo_Entrenado/metadata_modelo.txt'
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write("="*80 + "\n")
    f.write("METADATOS DEL MODELO DE DETECCIÓN DE FRAUDE\n")
    f.write("="*80 + "\n\n")
    f.write(f"Fecha de entrenamiento: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Tipo de modelo: XGBoost Classifier\n")
    f.write(f"Objetivo: binary:logistic\n")
    f.write(f"Métrica de evaluación: aucpr\n\n")
    f.write("Parámetros del modelo:\n")
    f.write(f"  - max_depth: {xgb_model.max_depth}\n")
    f.write(f"  - learning_rate: {xgb_model.learning_rate}\n")
    f.write(f"  - n_estimators: {xgb_model.n_estimators}\n")
    f.write(f"  - subsample: {xgb_model.subsample}\n")
    f.write(f"  - colsample_bytree: {xgb_model.colsample_bytree}\n\n")
    f.write("Rendimiento en conjunto de prueba:\n")
    f.write(f"  - Accuracy: {accuracy:.4f}\n")
    f.write(f"  - Precision (Fraude): {precision:.4f}\n")
    f.write(f"  - Recall (Sensibilidad): {recall:.4f}\n")
    f.write(f"  - F1-Score: {f1:.4f}\n")
    f.write(f"  - AUC-ROC: {auc_roc:.4f}\n")
    f.write(f"  - AUC-PR: {auc_pr:.4f}\n\n")
    f.write(f"Total de features: {len(feature_names)}\n")
    f.write(f"Features: {', '.join(feature_names)}\n")

print(f"[OK] Metadatos guardados en: {metadata_path}")
print("\n" + "="*80)
print("MODELO Y ARTEFACTOS GUARDADOS EXITOSAMENTE")
print("="*80)


8. GUARDANDO MODELO Y SCALER
[OK] Modelo guardado en: Modelo_Entrenado/modelo_xgboost_fraude.pkl
[OK] Scaler guardado en: Modelo_Entrenado/scaler_robust.pkl
[OK] Nombres de features guardados en: Modelo_Entrenado/features_names.pkl
[OK] Metadatos guardados en: Modelo_Entrenado/metadata_modelo.txt

MODELO Y ARTEFACTOS GUARDADOS EXITOSAMENTE


In [11]:
# ============================================================================
# 9. CURVAS ROC Y PRECISION-RECALL
# ============================================================================
print("\n" + "="*80)
print("9. GENERANDO CURVAS DE EVALUACIÓN")
print("="*80)

# Curva ROC
fpr, tpr, thresholds_roc = roc_curve(y_test, y_pred_proba)
roc_auc = auc_roc

# Curva Precision-Recall
precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc_pr

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate (Especificidad)', fontsize=12)
axes[0].set_ylabel('True Positive Rate (Sensibilidad)', fontsize=12)
axes[0].set_title('Curva ROC', fontsize=14, fontweight='bold')
axes[0].legend(loc="lower right")
axes[0].grid(alpha=0.3)

# Precision-Recall Curve
axes[1].plot(recall_curve, precision_curve, color='darkorange', lw=2, 
             label=f'PR curve (AUC = {pr_auc:.4f})')
baseline = (y_test == 1).sum() / len(y_test)
axes[1].axhline(y=baseline, color='navy', lw=2, linestyle='--', label='Baseline (Random)')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Recall (Sensibilidad)', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Curva Precision-Recall', fontsize=14, fontweight='bold')
axes[1].legend(loc="lower left")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('graficos_eda/08_curvas_roc_pr.png', dpi=300, bbox_inches='tight')
plt.close()
print("[OK] Grafico 8 guardado: 08_curvas_roc_pr.png")


9. GENERANDO CURVAS DE EVALUACIÓN
[OK] Grafico 8 guardado: 08_curvas_roc_pr.png


In [12]:
# ============================================================================
# 10. VALIDACIÓN CRUZADA
# ============================================================================
print("\n" + "="*80)
print("10. VALIDACIÓN CRUZADA ESTRATIFICADA")
print("="*80)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Validación cruzada con AUC-PR
cv_scores_aucpr = cross_val_score(
    xgb_model, X_train_balanced, y_train_balanced, 
    cv=skf, scoring='average_precision', n_jobs=-1
)

# Validación cruzada con AUC-ROC
cv_scores_aucroc = cross_val_score(
    xgb_model, X_train_balanced, y_train_balanced, 
    cv=skf, scoring='roc_auc', n_jobs=-1
)

# Validación cruzada con F1
cv_scores_f1 = cross_val_score(
    xgb_model, X_train_balanced, y_train_balanced, 
    cv=skf, scoring='f1', n_jobs=-1
)

print(f"\nValidación Cruzada (5-fold):")
print(f"AUC-PR: {cv_scores_aucpr.mean():.4f} (+/- {cv_scores_aucpr.std() * 2:.4f})")
print(f"AUC-ROC: {cv_scores_aucroc.mean():.4f} (+/- {cv_scores_aucroc.std() * 2:.4f})")
print(f"F1-Score: {cv_scores_f1.mean():.4f} (+/- {cv_scores_f1.std() * 2:.4f})")


10. VALIDACIÓN CRUZADA ESTRATIFICADA

Validación Cruzada (5-fold):
AUC-PR: 1.0000 (+/- 0.0000)
AUC-ROC: 1.0000 (+/- 0.0000)
F1-Score: 0.9995 (+/- 0.0003)


In [13]:
# ============================================================================
# 11. RESUMEN FINAL Y RESULTADOS
# ============================================================================
print("\n" + "="*80)
print("11. RESUMEN FINAL")
print("="*80)

results_summary = {
    'Métrica': ['Accuracy', 'Precision (Fraude)', 'Recall (Sensibilidad)', 
                'Specificity', 'F1-Score', 'AUC-ROC', 'AUC-PR'],
    'Valor': [accuracy, precision, recall, specificity, f1, auc_roc, auc_pr]
}

results_df = pd.DataFrame(results_summary)
print("\n" + results_df.to_string(index=False))

# Guardar resultados
results_df.to_csv('Archivos_CSV/resultados_modelo.csv', index=False)
print("\n[OK] Resultados guardados en: Archivos_CSV/resultados_modelo.csv")

# Guardar importancia de características
feature_importance.to_csv('Archivos_CSV/importancia_features.csv', index=False)
print("[OK] Importancia de caracteristicas guardada en: Archivos_CSV/importancia_features.csv")

print("\n" + "="*80)
print("ANÁLISIS COMPLETO FINALIZADO")
print("="*80)
print("\nGráficos guardados en la carpeta 'graficos_eda/'")
print("Resultados del modelo guardados en 'Archivos_CSV/resultados_modelo.csv'")


11. RESUMEN FINAL

              Métrica    Valor
             Accuracy 0.999446
   Precision (Fraude) 0.998893
Recall (Sensibilidad) 1.000000
          Specificity 0.998892
             F1-Score 0.999446
              AUC-ROC 0.999980
               AUC-PR 0.999978

[OK] Resultados guardados en: Archivos_CSV/resultados_modelo.csv
[OK] Importancia de caracteristicas guardada en: Archivos_CSV/importancia_features.csv

ANÁLISIS COMPLETO FINALIZADO

Gráficos guardados en la carpeta 'graficos_eda/'
Resultados del modelo guardados en 'Archivos_CSV/resultados_modelo.csv'
